In [1]:
%pip install -q ibm-watson-machine-learning
%pip install -q ibm-watsonx-ai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ibm_watson_machine_learning import APIClient
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.client import APIClient as WatsonxAPIClient

# --- PASTE YOUR CREDENTIALS HERE ---
API_KEY = "CogOqUMsBHlotDis07P5UG0vkANcsB7JNCtCzTRPMkZ6"
PROJECT_ID = "71124e53-992d-46ce-8e3b-1ade2c10caba"
SPACE_ID = "f210a915-20f4-462f-a4b3-3ca197b32a41"

DIABETES_DEPLOYMENT_ID = "4a16a588-86ae-40e9-92a8-bfe85957d205"
HEART_DEPLOYMENT_ID = "1350a17d-e7c5-4c1f-9ca7-d5f173c4dcfd"
HYPERTENSION_DEPLOYMENT_ID = "8b6ab145-9d92-47ae-8a73-408a7b09a57a"
# ------------------------------------

# Define the correct URL for the London region
LONDON_URL = "https://eu-gb.ml.cloud.ibm.com"

# Setup the client for calling the deployed ML models
wml_credentials = {"url": LONDON_URL, "apikey": API_KEY}
wml_client = APIClient(wml_credentials)
wml_client.set.default_space(SPACE_ID)

# Setup the client for calling the Granite LLM model
watsonx_credentials = {
    "url": LONDON_URL,
    "apikey": API_KEY
}
watsonx_client = WatsonxAPIClient(credentials=watsonx_credentials, project_id=PROJECT_ID)

print("Authentication successful!")

Authentication successful!


In [3]:
# This is the data for a single patient
diabetes_input = {
    "input_data": [{
        "fields": ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"],
        "values": [[6, 148, 72, 35, 0, 33.6, 0.627, 50]]
    }]
}

heart_disease_input = {
    "input_data": [{
        "fields": ["id", "age", "sex", "dataset", "cp", "trestbps", "chol", "fbs", "restecg", "thalch", "exang", "oldpeak", "slope", "ca", "thal"],
        "values": [[1, 52, "Male", "Cleveland", "non-anginal", 130, 204, "FALSE", "normal", 172, "FALSE", 1.4, "upsloping", 0, "normal"]]
    }]
}
hypertension_input = {
    "input_data": [{
        "fields": ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"],
        "values": [[56, 1, 0, 120, 236, 0, 1, 178, 0, 0.8, 2, 0, 2]]
    }]
}

print("Patient data is ready.")

Patient data is ready.


In [4]:
import time
from ibm_watson_machine_learning.wml_client_error import ApiRequestFailure

# Helper function to call a deployment with retries
def score_with_retry(deployment_id, input_data, model_name):
    retries = 3
    for i in range(retries):
        try:
            print(f"Attempting to score {model_name} model (Attempt {i+1}/{retries})...")
            prediction_response = wml_client.deployments.score(deployment_id, input_data)
            print(f"   -> {model_name} scoring successful!")
            return prediction_response
        except ApiRequestFailure as e:
            print(f"   -> WARNING: {model_name} model failed with error: {e.status_code}. Retrying in 5 seconds...")
            if i < retries - 1: # Don't sleep on the last attempt
                time.sleep(5)
            else:
                print(f"   -> ERROR: {model_name} model failed after all retries. Raising error.")
                raise e # Re-raise the exception if all retries fail

# --- Call each deployed model using the robust retry function ---
print("Getting predictions from AI models with retry logic...")

# Call each deployed model
diabetes_prediction_response = score_with_retry(DIABETES_DEPLOYMENT_ID, diabetes_input, "Diabetes")
heart_prediction_response = score_with_retry(HEART_DEPLOYMENT_ID, heart_disease_input, "Heart Disease")
hypertension_prediction_response = score_with_retry(HYPERTENSION_DEPLOYMENT_ID, hypertension_input, "Hypertension")

# --- Extract the actual prediction (0 or 1) from the response ---
diabetes_result = diabetes_prediction_response['predictions'][0]['values'][0][0]
heart_result = heart_prediction_response['predictions'][0]['values'][0][0]
hypertension_result = hypertension_prediction_response['predictions'][0]['values'][0][0]

print("-" * 30)
print("All models responded successfully!")
print(f"--> Diabetes Prediction: {diabetes_result} (1=High Risk, 0=Low Risk)")
print(f"--> Heart Disease Prediction: {heart_result} (1=High Risk, 0=Low Risk)")
print(f"--> Hypertension Prediction: {hypertension_result} (1=High Risk, 0=Low Risk)")
print("-" * 30)

Getting predictions from AI models with retry logic...
Attempting to score Diabetes model (Attempt 1/3)...
   -> Diabetes scoring successful!
Attempting to score Heart Disease model (Attempt 1/3)...
   -> Heart Disease scoring successful!
Attempting to score Hypertension model (Attempt 1/3)...
   -> Hypertension scoring successful!
------------------------------
All models responded successfully!
--> Diabetes Prediction: 1.0 (1=High Risk, 0=Low Risk)
--> Heart Disease Prediction: 0.0 (1=High Risk, 0=Low Risk)
--> Hypertension Prediction: 1.0 (1=High Risk, 0=Low Risk)
------------------------------


In [5]:
# Convert numeric predictions to human-readable text
diabetes_risk_text = "High" if diabetes_result == 1 else "Low"
heart_risk_text = "High" if heart_result == 1 else "Low"
hypertension_risk_text = "High" if hypertension_result == 1 else "Low"

# This is the master prompt template you designed
prompt_template = f"""
You are an AI Health Assistant. Your job is to provide a single, consolidated, and supportive message to a user based on their health risk factors.

**Instructions:**
1. Review all three risk levels: Diabetes, Heart Condition, and Hypertension.
2. Identify any risk that is "High".
3. If one risk is "High", make it the main topic of your response.
4. If multiple risks are "High", choose Heart Condition as the primary topic, then mention the others.
5. If all risks are "Low", provide a general wellness tip.
6. For each "High" risk, provide one simple, actionable lifestyle tip.
7. The tone must be calm, supportive, and never alarming. Do not use medical jargon.
8. **Crucially, always end the response by strongly recommending that the user consult a real doctor for personalized medical advice.**

**User's Health Data Analysis:**
- Diabetes Risk: {diabetes_risk_text}
- Heart Condition Risk: {heart_risk_text}
- Hypertension Risk: {hypertension_risk_text}

**AI Assistant's Response:**
"""

# Configure the Granite model
model_params = {"decoding_method": "greedy", "max_new_tokens": 300, "min_new_tokens": 50}
granite_model = Model("ibm/granite-13b-instruct-v2", watsonx_credentials, project_id=PROJECT_ID)

print("Generating personalized advice from the AI Health Agent...")
print("-" * 50)

# Call the Granite model and get the response
generated_response = granite_model.generate_text(prompt=prompt_template, params=model_params)

# Print the final advice
print(generated_response)

/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/model.py:105: DeprecationWarning: The `Model` class is deprecated and will be removed in a future release. Please use the `ModelInference` class instead. To update your imports, use: `from ibm_watsonx_ai.foundation_models import ModelInference`.
  warn(model_class_deprecated_warning, category=DeprecationWarning)
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'ibm/granite-13b-instruct-v2' is in deprecated state from 2025-06-18 until 2025-10-15. IDs of alternative models: ibm/granite-3-3-8b-instruct. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


Generating personalized advice from the AI Health Agent...
--------------------------------------------------
Hello, I have reviewed your health data and I noticed that you have a high risk of diabetes and hypertension. I would like to recommend that you consult a doctor for personalized medical advice. You can also try the following lifestyle tips to help reduce your risk of diabetes and hypertension:
- Get at least 7 hours of sleep each night.
- Eat a healthy diet that is low in sugar and salt.
- Exercise for at least 30 minutes every day.
- Limit your alcohol intake.
- Take your medication as prescribed.
- Avoid smoking.
- Manage your stress levels.
If you have any further questions, please let me know. I hope this information is helpful.



In [6]:
def generate_medication_reminder(patient_risks):
    reminders = []
    if patient_risks['diabetes'] == 'High':
        reminders.append("- Take your Metformin after your morning meal as prescribed.")
    if patient_risks['hypertension'] == 'High':
        reminders.append("- Remember to take your blood pressure medication (e.g., Lisinopril) every day.")
    if not reminders:
        return "No specific medication reminders are set based on your current risk profile."
        
    reminder_text = "Here are your personalized medication reminders:\n" + "\n".join(reminders)
    reminder_text += "\n\nPlease note: This is a sample reminder. Always follow the specific instructions from your doctor."
    return reminder_text

# Create a dictionary of the patient's risks
current_risks = {
    'diabetes': diabetes_risk_text,
    'hypertension': hypertension_risk_text,
    'heart': heart_risk_text
}

print("--- Medication Reminder Feature ---")
medication_message = generate_medication_reminder(current_risks)
print(medication_message)

--- Medication Reminder Feature ---
Here are your personalized medication reminders:
- Take your Metformin after your morning meal as prescribed.
- Remember to take your blood pressure medication (e.g., Lisinopril) every day.

Please note: This is a sample reminder. Always follow the specific instructions from your doctor.


In [7]:
print("="*60)
print("      *** AI AGENT'S COMPLETE PROACTIVE CARE PLAN ***")
print("="*60)

print("\n--- DATA SOURCE SIMULATION ---")
print("This agent has analyzed health data from a combination of sources:")
print("  - Wearable Device (simulated heart rate, activity levels)")
print("  - Medical Records (simulated cholesterol, blood pressure)")
print("  - Patient Input (simulated age, family history)")

print("\n--- EARLY WARNING SIGN DETECTION ---")
print(f"  - Diabetes Risk Assessment: {diabetes_risk_text}")
print(f"  - Heart Condition Risk Assessment: {heart_risk_text}")
print(f"  - Hypertension Risk Assessment: {hypertension_risk_text}")

print("\n--- PERSONALIZED INSIGHTS & LIFESTYLE RECOMMENDATIONS ---")
# We are re-printing the advice from the Granite model for clarity
print("Based on this analysis, the AI agent provides the following guidance:")
print('"' + generated_response.strip() + '"')

print("\n--- PERSONALIZED MEDICATION REMINDERS ---")
print(medication_message)

print("\n--- OUTCOME: PROACTIVE CARE ENABLED ---")
print("By providing these early warnings and actionable steps, this AI agent")
print("enables proactive care, helping to reduce hospital visits and improve")
print("long-term health outcomes by bridging the gap between patient and provider.")
print("="*60)


      *** AI AGENT'S COMPLETE PROACTIVE CARE PLAN ***

--- DATA SOURCE SIMULATION ---
This agent has analyzed health data from a combination of sources:
  - Wearable Device (simulated heart rate, activity levels)
  - Medical Records (simulated cholesterol, blood pressure)
  - Patient Input (simulated age, family history)

--- EARLY WARNING SIGN DETECTION ---
  - Diabetes Risk Assessment: High
  - Heart Condition Risk Assessment: Low
  - Hypertension Risk Assessment: High

--- PERSONALIZED INSIGHTS & LIFESTYLE RECOMMENDATIONS ---
Based on this analysis, the AI agent provides the following guidance:
"Hello, I have reviewed your health data and I noticed that you have a high risk of diabetes and hypertension. I would like to recommend that you consult a doctor for personalized medical advice. You can also try the following lifestyle tips to help reduce your risk of diabetes and hypertension:
- Get at least 7 hours of sleep each night.
- Eat a healthy diet that is low in sugar and salt.
- 